In [ ]:
print('True')

In [2]:
# importing os and load_dotenv
import os
from dotenv import load_dotenv

In [3]:
# loading dotenv and getting the openai api key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# getting the openai key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
# importing PyPDFLoader from langchain.document_loaders
from langchain.document_loaders import PyPDFLoader

In [6]:
# showing the present working directory
%pwd

'd:\\genai_project\\Interview-Questions-Generator\\research'

In [ ]:
%cd ..

In [8]:
%pwd

'd:\\genai_project\\Interview-Questions-Generator'

# Step 1: Ingesting Data

In [9]:
# path of the file daved into a variable
# calling the PyPDFLoader() class and creating an object
# and through the object call a function call functioned as load()
file_path = 'data/SDG.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()

In [ ]:
# showing the whole dataset
data
# showing the data through indexing
data[0]
# length of the dataset
len(data)
# printint the dataset through length wise
for i in data:
    print(i.page_content)

In [14]:
# saving the all page content of the whole dataset into a string varible
question_gen = ''

for page in data:
    question_gen += page.page_content

# Step-2 : Chunking 
#### means partioning text into chunking giving chunk_size and chunk_overlap

In [15]:
# importing TokenTextSplitter from langchain.text_splitter. we need it for
# splitting the whole documents because every model has specific token limitations
# or we can say chunk_size
from langchain.text_splitter import TokenTextSplitter

In [18]:
# now we will call the TokenTextSplitter() class and making an object of it
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 10000,
    chunk_overlap = 200
)

In [19]:
# calling the function split_text() through the object we just created
# and passing the question_gen where all data are saved into single string varible.
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)
# now our whole dataset saved into a list inside the string type data

In [30]:
# importing Document from langchain.docstore.document
from langchain.docstore.document import Document

In [33]:
# now we need to convert the data into documets type
# and this is the procedure to convert the data into a list
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]


In [32]:
# creating an obejct of TokenTextSplitter() class
# why we are doing the same thing because if we check the previous splitter,
# the length of the list is 1
# and we use 10000 for chunk size.
# now we will take smaller chunks size and splitting the text
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [34]:
# calling split_documents() function through the object splitter_ans_gen
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [38]:
# getting first index of the documents through the page_content attribute
document_answer_gen[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [39]:
# length of the document
len(document_answer_gen)

4